In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tqdm import tqdm

In [2]:
TRAIN_IMAGES_PATH = '/Volumes/SSD/rsna-2022-cervical-spine-fracture-detection/train_images'
TEST_IMAGES_PATH = '/Volumes/SSD/rsna-2022-cervical-spine-fracture-detection/test_images'
METADATA_PATH = '/Volumes/SSD/rsna-2022-cervical-spine-fracture-detection/metadata'

In [3]:
import pandas as pd
df_seg = pd.read_csv(f'{METADATA_PATH}/meta_segmentation.csv')

In [4]:
slice_max_seg = df_seg.groupby('StudyInstanceUID')['Slice'].max().to_dict()
df_seg['SliceRatio'] = 0
df_seg['SliceRatio'] = df_seg['Slice'] / df_seg['StudyInstanceUID'].map(slice_max_seg)

In [9]:
unique_list = list(df_seg['StudyInstanceUID'].unique())

In [20]:
print(unique_list)

['1.2.826.0.1.3680043.10633', '1.2.826.0.1.3680043.10921', '1.2.826.0.1.3680043.11827', '1.2.826.0.1.3680043.11988', '1.2.826.0.1.3680043.12281', '1.2.826.0.1.3680043.12292', '1.2.826.0.1.3680043.12833', '1.2.826.0.1.3680043.1363', '1.2.826.0.1.3680043.14267', '1.2.826.0.1.3680043.1480', '1.2.826.0.1.3680043.15206', '1.2.826.0.1.3680043.1542', '1.2.826.0.1.3680043.1573', '1.2.826.0.1.3680043.16092', '1.2.826.0.1.3680043.16919', '1.2.826.0.1.3680043.17481', '1.2.826.0.1.3680043.17960', '1.2.826.0.1.3680043.18480', '1.2.826.0.1.3680043.1868', '1.2.826.0.1.3680043.18906', '1.2.826.0.1.3680043.18935', '1.2.826.0.1.3680043.18968', '1.2.826.0.1.3680043.19021', '1.2.826.0.1.3680043.19333', '1.2.826.0.1.3680043.19388', '1.2.826.0.1.3680043.20120', '1.2.826.0.1.3680043.20647', '1.2.826.0.1.3680043.20928', '1.2.826.0.1.3680043.21321', '1.2.826.0.1.3680043.21651', '1.2.826.0.1.3680043.2243', '1.2.826.0.1.3680043.23904', '1.2.826.0.1.3680043.24140', '1.2.826.0.1.3680043.24606', '1.2.826.0.1.368004

In [46]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [47]:
gauth = GoogleAuth()      
drive = GoogleDrive(gauth)  

In [48]:
file_metadata = {
  'title': 'test_folder',
  'parents' : [{'id': '13s1wn9aXcItw-LefF2BGPWqNWVEl4mni'}],
  'mimeType': 'application/vnd.google-apps.folder'
}

folder = drive.CreateFile(file_metadata)
folder.Upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1070504942186-iqh4fq2ks7lvgcap6mgqtne68hhohbfu.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [49]:
file_list = drive.ListFile({'q': "'13s1wn9aXcItw-LefF2BGPWqNWVEl4mni' in parents and trashed=false"}).GetList()
already_in_folder = []
for i in file_list:
    already_in_folder.append(i['title'])
print(already_in_folder)

['1.2.826.0.1.3680043.26979', '1.2.826.0.1.3680043.26898', '1.2.826.0.1.3680043.26740', '1.2.826.0.1.3680043.26498', '1.2.826.0.1.3680043.26492', '1.2.826.0.1.3680043.26442', '1.2.826.0.1.3680043.26110', '1.2.826.0.1.3680043.26068', '1.2.826.0.1.3680043.25833', '1.2.826.0.1.3680043.25704', '1.2.826.0.1.3680043.24891', '1.2.826.0.1.3680043.24617', '1.2.826.0.1.3680043.24606', '1.2.826.0.1.3680043.24140', '1.2.826.0.1.3680043.23904', '1.2.826.0.1.3680043.2243', '1.2.826.0.1.3680043.21651', '1.2.826.0.1.3680043.21321', '1.2.826.0.1.3680043.20928', '1.2.826.0.1.3680043.20647', '1.2.826.0.1.3680043.20120', '1.2.826.0.1.3680043.19388', '1.2.826.0.1.3680043.19333', '1.2.826.0.1.3680043.19021', '1.2.826.0.1.3680043.18968', '1.2.826.0.1.3680043.18935', '1.2.826.0.1.3680043.18906', '1.2.826.0.1.3680043.1868', '1.2.826.0.1.3680043.18480', '1.2.826.0.1.3680043.17960', '1.2.826.0.1.3680043.17481', '1.2.826.0.1.3680043.16919', '1.2.826.0.1.3680043.16092', '1.2.826.0.1.3680043.1573', '1.2.826.0.1.368

In [50]:
for i in already_in_folder:
    if i in unique_list:
        unique_list.remove(i)
print(unique_list)

['1.2.826.0.1.3680043.26990', '1.2.826.0.1.3680043.27016', '1.2.826.0.1.3680043.27292', '1.2.826.0.1.3680043.27752', '1.2.826.0.1.3680043.28025', '1.2.826.0.1.3680043.28327', '1.2.826.0.1.3680043.28665', '1.2.826.0.1.3680043.29425', '1.2.826.0.1.3680043.30067', '1.2.826.0.1.3680043.30487', '1.2.826.0.1.3680043.30524', '1.2.826.0.1.3680043.30565', '1.2.826.0.1.3680043.30640', '1.2.826.0.1.3680043.31077', '1.2.826.0.1.3680043.3168', '1.2.826.0.1.3680043.32071', '1.2.826.0.1.3680043.32280', '1.2.826.0.1.3680043.32370', '1.2.826.0.1.3680043.32434', '1.2.826.0.1.3680043.32436', '1.2.826.0.1.3680043.32590', '1.2.826.0.1.3680043.32658', '1.2.826.0.1.3680043.3376', '1.2.826.0.1.3680043.3882', '1.2.826.0.1.3680043.3992', '1.2.826.0.1.3680043.4202', '1.2.826.0.1.3680043.4769', '1.2.826.0.1.3680043.5002', '1.2.826.0.1.3680043.5671', '1.2.826.0.1.3680043.5782', '1.2.826.0.1.3680043.5783', '1.2.826.0.1.3680043.6078', '1.2.826.0.1.3680043.6125', '1.2.826.0.1.3680043.6376', '1.2.826.0.1.3680043.780',

In [51]:
# file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('1cIMiqUDUNldxO6Nl-KVuS9SV-cWi9WLi')}).GetList()
# for file in file_list:
# 	print('title: %s, id: %s' % (file['title'], file['id']))
parent_folder = '13s1wn9aXcItw-LefF2BGPWqNWVEl4mni'


for i in unique_list:
    file_metadata = {
        'title': i,
        'parents' : [{'id': parent_folder}],
        'mimeType': 'application/vnd.google-apps.folder'
    }
    folder = drive.CreateFile(file_metadata)
    folder.Upload()
    
    child_folder_id = ''
    file_list = drive.ListFile({'q': "'13s1wn9aXcItw-LefF2BGPWqNWVEl4mni' in parents and trashed=false"}).GetList()
    for j in file_list:
        if j['title'] == i:
            child_folder_id = j['id']
            break
        
    path = "/Volumes/SSD/rsna-2022-cervical-spine-fracture-detection/train_images/" + str(i)   
    
    # iterating thought all the files/folder
    # of the desired directory
    for x in os.listdir(path):
    
        f = drive.CreateFile({'parents' : [{'id': child_folder_id}], 'title': x})
        f.SetContentFile(os.path.join(path, x))
        f.Upload()
    
        # Due to a known bug in pydrive if we 
        # don't empty the variable used to
        # upload the files to Google Drive the
        # file stays open in memory and causes a
        # memory leak, therefore preventing its 
        # deletion
        f = None
# file1 = drive.CreateFile({'parents' : [{'id': folder}], 'title': x})


KeyboardInterrupt: 

In [6]:
targets = ['C1','C2','C3','C4','C5','C6','C7']
print(len(df_seg))

29832


In [7]:
y = df_seg[targets].to_numpy()

In [8]:
df_seg_1 = df_seg[:5000]

In [9]:
input_arr = []
for i in df_seg_1.index:
    study_instance = df_seg_1['StudyInstanceUID'][i]
    instance_slice = df_seg_1['Slice'][i]
    path_img = f'/Volumes/SSD/rsna-2022-cervical-spine-fracture-detection/class_train/classification_training/{study_instance}-{instance_slice}.png'
    image = tf.keras.preprocessing.image.load_img(path_img)
    input_arr.append(tf.keras.preprocessing.image.img_to_array(image))
print('done')

done


In [10]:
input_arr = np.array([input_arr]) 

: 

: 

In [8]:
# from keras.preprocessing.image import ImageDataGenerator
# batches = ImageDataGenerator().flow_from_directory(directory='/Volumes/SSD/rsna-2022-cervical-spine-fracture-detection/class_train', target_size=(512,512), batch_size=20000)
# imgs, labels = next(batches)
# ms = np.array(imgs[0]).astype(np.uint8)
# plt.imshow(ms)